In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import random
import pandas as pd
from selenium.webdriver import ChromeOptions
import datetime

In [2]:
# company_list=pd.read_csv('Oceans Companies to Track in Past 90 Days.csv')
# company_list=company_list['Name']
company_list=pd.read_excel('companyList.xlsx')
company_list=company_list['company name']

keywords_list=pd.read_excel('keywordList.xlsx')
keywords=keywords_list['keywords']

columns=keywords.tolist()
columns.insert (0, "Content")
columns.insert (0, "Link")
columns.insert (0, "Time")
columns.insert (0, "Title")
columns.insert (0, "Source")

# df=pd.read_excel('BusinessWire_'+str(datetime.date.today())+'.xlsx',sheet_name='amazon')
# df

## BusinessWire

In [3]:
def businessWire(company,keywords,columns=columns):
    path_of_chromedriver = r'./chromedriver' 
    driver = webdriver.Chrome(path_of_chromedriver)
    wait = WebDriverWait(driver=driver, timeout=300)
    def search(company):
        driver.get('https://www.businesswire.com/portal/site/home/my-business-wire/')
        wait.until(
            EC.presence_of_element_located((By.ID,'bw-search-input'))
        )

        search = driver.find_element_by_id('bw-search-input')
        search.clear()
        search.send_keys(str(company))
        search.send_keys(Keys.RETURN)

        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME,'bw-news-section'))
        )
        titles = []
        links = []
        times = []
        for i in range(1,6):
            ele = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div/div[3]/section[2]/ul/li[{}]/h3/a'.format(i))
            title = ele.text
            titles.append(title)
            link = ele.get_attribute('href')
            links.append(link)
            posted_time = driver.find_element_by_xpath('//*[@id="bw-group-all"]/div/div/div[3]/section[2]/ul/li[{}]/div[1]/time'.format(i)).text
            times.append(posted_time)
        return(titles, times, links)


    def get_content(links):
        contents = []
        for link in links:
            driver.get(link)
            content = driver.find_element_by_class_name('bw-release-body  ').text
            contents.append(content)
    #     print(contents)
        return(contents)

    def df_transform(df):
        for i in keywords:
            l=[]
            for m in range(len(df.index)):
                l2=df.iloc[m]['Content'].split("\n")
                s=""
                for z in l2:
                    if i in z:
                        s+=z
                if s=="":
                    l.append('None')
                else:
                    l.append(s)

            df[i]=l
        return df
#     writer = pd.ExcelWriter('BusinessWire_'+str(datetime.date.today())+'.xlsx')
#     Sheet_list.append('BusinessWire_'+str(datetime.date.today())+'.xlsx')
    try:
        titles, times, links = search(company)
        contents = get_content(links)
        source=['BusinessWire'] * len(times)
        result_dict = {'Source':source,
                    'Title': titles,
                   'Time': times,
                  'Link': links,
                  'Content': contents}
        result_pd = pd.DataFrame(result_dict)
        df_BusinessWire=df_transform(result_pd)
    except:
        df_BusinessWire = pd.DataFrame(columns=columns)
    driver.quit()

    return df_BusinessWire
#         writer.save()
#         writer.close()
    

# Prnewswire

In [4]:
def prnewsWire(company,keywords,columns=columns):
    path_of_chromedriver = r'./chromedriver' 
    driver = webdriver.Chrome(path_of_chromedriver)
    wait = WebDriverWait(driver=driver, timeout=300)
    def search(company):
        driver.get('https://www.prnewswire.com/search/all/?keyword='+str(company))

        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME,'row'))
        )

        titles = []
        links = []
        times = []

        items = driver.find_elements_by_class_name('news-release')
        item_num = len(items)
        if item_num == 0:
            titles = ['Nothing Found']
            return(titles, links)
        else:
            for item in items:
                title = item.text
                titles.append(title)

                link = item.get_attribute('href')
                links.append(link)
        return(titles, links)

    def df_transform(df):
        for i in keywords:
            l=[]
            for m in range(len(df.index)):
                l2=df.iloc[m]['Content'].split("\n")
                s=""
                for z in l2:
                    if i in z:
                        s+=z
                if s=="":
                    l.append('None')
                else:
                    l.append(s)

            df[i]=l
        return df
    def get_content(links):
        contents = []
        times = []
        for link in links:
            driver.get(link)
            time = driver.find_element_by_class_name('mb-no').text
            times.append(time)
            content = driver.find_element_by_xpath('//*[@id="main"]/article/section/div/div').text
            contents.append(content)
        return(times, contents)
    try:
        company=company.replace(' ','%20')
        titles, links = search(company)
        times, contents = get_content(links)
        source=['Prnewswire']*len(times)
        result_dict = {'Source':source,
                    'Title': titles,
                    'Time': times,
                    'Link': links,
                    'Content': contents}
        df_Prnewswire = pd.DataFrame(result_dict)
        df_Prnewswire=df_transform(df_Prnewswire)
        company=company.replace('%20',' ')
        driver.quit()
    except:
        df_Prnewswire = pd.DataFrame(columns=columns)
    return df_Prnewswire

# Product Hunt

In [5]:
def productHunt(company,keywords,columns=columns):
    path_of_chromedriver = r'./chromedriver' 
    driver = webdriver.Chrome(path_of_chromedriver)
    wait = WebDriverWait(driver=driver, timeout=300)
    def search(company):
        driver.get('https://www.producthunt.com/search?q='+str(company))
        titles = []
        links = []
        for index in range(1,11):
            try:
                ele = driver.find_element_by_xpath(f'//*[@id="__next"]/div[2]/main/div[2]/div/div/div[{index}]/div[1]/div/h3/a[1]')
            except:
                continue
            title = ele.text
            titles.append(title)
            link = ele.get_attribute('href')
            links.append(link)
        return(titles, links)


    def get_content(links):
        contents = []
        for link in links:
            driver.get(link)
            try:
                content = driver.find_element_by_xpath('//*[@id="__next"]/main/div[2]/div/section/div/div[1]/div/div/p').text
                contents.append(content)
            except:
                contents.append('')
    #     print(contents)
        return(contents)
    def df_transform(df):
        for i in keywords:
            l=[]
            for m in range(len(df.index)):
                l2=df.iloc[m]['Content'].split("\n")
                s=""
                for z in l2:
                    if i in z:
                        s+=z
                if s=="":
                    l.append('None')
                else:
                    l.append(s)

            df[i]=l
        return df
    try:
        search(company)
        titles, links = search(company)
        contents = get_content(links)
        source=['ProductHunt']*len(titles)
        times=[str(datetime.date.today())]*len(titles)
        result_dict = {'Source':source,
                      'Title': titles,
                      'Time':times,
                      'Link': links,
                      'Content': contents}
        df_ProductHunt = pd.DataFrame(result_dict)
        df_ProductHunt=df_transform(df_ProductHunt)
        driver.quit()
    except:
        df_ProductHunt = pd.DataFrame(columns=columns)
    return df_ProductHunt

# Venturebeat

In [6]:
def ventureBeat(company,keywords,columns=columns):
    path_of_chromedriver = r'./chromedriver' 
    driver = webdriver.Chrome(path_of_chromedriver)
    wait = WebDriverWait(driver=driver, timeout=300)
    def search(company):
        driver.get('https://venturebeat.com/?s='+str(company))
        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME,'page-title'))
        )

        titles = []
        links = []
        times = []

        items = driver.find_elements_by_class_name('ArticleListing__body')
        item_num = len(items)
        if item_num == 0:
            return('Nothing found')
        else:
            i = 1
            while i < 7:
                ele = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/article[{}]/header/h2/a'.format(i))
                title = ele.text.strip()
                titles.append(title)
                link = ele.get_attribute('href')
                links.append(link)
                posted_time = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/article[{}]/header/div[2]/time'.format(i)).text
                times.append(posted_time)
                i+=1
            return(titles, times, links)

    def get_content(links):
        contents = []
        for link in links:
            driver.get(link)
            content = driver.find_element_by_class_name('article-content').text
            modified_content = []
            parsed_content = content.split('\n')
            length = len(parsed_content)-1
            for i in range(1,length):
                c = parsed_content[i]
                modified_content.append(c)
            content = "\n".join(modified_content)
            contents.append(content)
        return(contents)
    def df_transform(df):
        for i in keywords:
            l=[]
            for m in range(len(df.index)):
                l2=df.iloc[m]['Content'].split("\n")
                s=""
                for z in l2:
                    if i in z:
                        s+=z
                if s=="":
                    l.append('None')
                else:
                    l.append(s)

            df[i]=l
        return df
    try:
        search(company)
        titles, times, links = search(company)
        contents = get_content(links)
        source=['Venturebeat']*len(titles)
        result_dict = {'Source':source,
                    'Title': titles,
                   'Time': times,
                  'Link': links,
                  'Content': contents}
        df_Venturebeat = pd.DataFrame(result_dict)
        driver.quit()
    except:
        df_Venturebeat = pd.DataFrame(columns=columns)
    return df_Venturebeat


In [7]:
writer = pd.ExcelWriter('Target Company Info-'+str(datetime.date.today())+'.xlsx')
for company in company_list:
    if len(company)>=30:
        company=company[:30]
    df1=businessWire(company,keywords)
    df2=prnewsWire(company,keywords)
    df3=productHunt(company,keywords)
    df4=ventureBeat(company,keywords)
    df=pd.concat([df1,df2,df3,df4])
    df.to_excel(writer, sheet_name=company,index=False)
writer.save()
writer.close()

# Send Email

In [8]:
import smtplib
import datetime
import email
# text
from email.mime.text import MIMEText
# image
from email.mime.image import MIMEImage
#assemble
from email.mime.multipart import MIMEMultipart
from email.header import Header
mail_host = "smtp.163.com"
# sender
mail_sender = "oceansCrawler@163.com"

mail_license = "RJXQCJKXYJUCJERM"
# receiver
mail_receivers = ["haijiang@wustl.edu"]
mm = MIMEMultipart('related')
# subject
subject_content = """Oceans Crawler """+str(datetime.date.today())
# sender
mm["From"] = "Crawler Robot<oceansCrawler@163.com>"
# receiver
mm["To"] = "receiver_1_name<haijiang@wustl.edu>"
# Subject
mm["Subject"] = Header(subject_content,'utf-8')
# content
body_content = """Hello，attanched is the data crawled！"""
# param1：contene，param2：text type，param3：code tyoe
message_text = MIMEText(body_content,"plain","utf-8")

mm.attach(message_text)

filename='Target Company Info-'+str(datetime.date.today())+'.xlsx'
# create att
atta = MIMEText(open(filename, 'rb').read(), 'base64', 'utf-8')
# set attachment
atta["Content-Disposition"] = 'attachment; filename='+filename
# add attachment
mm.attach(atta)

# create STMP object
stp = smtplib.SMTP()
# setter
stp.connect(mail_host, 25)  

# login email，parm1：email adderss，param：authorization code
stp.login(mail_sender,mail_license)
# send email，param1：sender address，param2：receiver address，parms3：change content datatyoe to str
stp.sendmail(mail_sender, mail_receivers, mm.as_string())
print("sent successfully")
# shut down SMTP object
stp.quit()

sent successfully


(221, b'Bye')